# 24) HW3 solution

## Part 1: Optimizing dot product (30%)

(5%) The compile/runtime unrolling factor versions achieve performance similar to the `dot_opt2` version, but not as good as the `dot_opt3` version. 

In addition to your commentary, answer the following questions:

1. (5%) Is this code correct for all values of n?
    - No, for odd `n`, we incur in out-of-bound access when we perform the second product, `a[i+1] * b[i+1]`.
2. (5%) Can this code change the numerically computed result?
    - Yes, because of floating-point aritmethic (FPA). We're adding an extra operation. If any of the numbers to add were exrtemely small (or large), we will suffer from roundoff errors. Also, as you've learned, the C standard does not allow floating point arithmetic to be reordered because it may change the computed values (non associativity).
3. (5%) Can you extend this technique to increase performance further (longer vector registers and/or more instruction-level parallelism)?
    - Yes, we can definitely consider instead of an unrolling factor 2, chunking the data in larger sizes and apply SIMD-like operations (like we do in Part 2)
4. (5%) Can you make the unrolling factor 2 a compile-time constant, perhaps by using an inner loop?
    - Yes, we can do that (see `dot_opt_comp_time_m` function)
5. (5%) Could that unrolling factor be a run-time parameter?
    - Yes, we can do that (see `dot_opt_run_time_m` function).
    There are a couple of different ways this can be achieved. 
      * If someone defined the runtime parameter, in say, `main`, then the compiler would get "smart" and optimize and treat it in the same way as a compile-time constant. Keep in mind that we're compiling with the optimization flag `-O3`, so if we only used that unrolling factor in the call to our function in `main`, then the the compiler knows at compile time the value that we're passing, and because we're calling that function only once, it might just substitute the runtime constant as it were a constant known at compile time
      
      * But if someone actually added it as a true runtime unrolling factor, passed via a command-line argument, which the compiler cannot know in advance, then, yes, the performance of the runtime version would be lower than the compile-time counterpart.  
    

In [ ]:
! gcc -O3 -march=native -fopenmp    dot.c   -o dot

In [ ]:
! OMP_NUM_THREADS=4 ./dot -r 10 -n 10000

## Part 2: Optimizing block inner product (%70)

- What does it mean to reorder loops? Will it help or hurt performance?


The two lines that originally use the `j` as slowest index and `k` as fastest index to perform the pair-wise products:

```
for (size_t j=0; j<J; j++) {
    for (size_t k=0; k<K; k++) {
```

tell us that the same row of the matrix $A^T$ is multiplied by all columns of $B$, and we store the result in $C$ row-wise. 

If we swapped these two lines, (see the `bdot_opt3_simd_reordered` function where we do this), we would multiply the same column of $B$ by all rows of $A^T$ and we'd store the result in $C$ column-wise. 

The effect of doing so it is actually negligible on performance (on my architecture) and might bring only a very minimal advantage on some other architectures. If you think about it, this would only change how 32 pair-wise dot products are performed, but the majority of the work in this code is done in the single pair-wise dot products (20000 FLOPs each).

- Does it help to change the layout in memory (see the `aistride`, `ajstride`, `bistride`, and `bkstride` parameters in the code)?

No, doing it without further algorithmic changes in the functions where we actually perform the dot products, does not help. First of all, the only way the stride parameters can be changed is in the call of the `init_bdot` function. Instead of calling `init_bdot` as we called it originally:

```
init_bdot(args.length, a, n, 1, b, 1, n);
```

one of the two arguments between `n` and `1` has to be kept to `1` to fill the whole matrices `A` and `B` without any gaps, as in:

```
init_bdot(args.length, a, 1, n, b, n, 1);
```

This would make `A` be stored column-wise and `B` stored row-wise (effectively transposing the matrices). If we did this, in order for the algorithm to still produce the same results of the pairwise dot products, we would also need to perform further algorithmic changes. Even if we did perform those further algorithmic changes to make the results correct, it would not help performance, as we would be back to the original situation.

One might have also thought to use smaller blocks/stride patterns, such as `1, J` and `1, K`. However, again, this would only change how the data entries are laid out, and without further algorithmic changes in the functions that actually call the pair-wise dot products (which are implemented to perform the dot product of the _entire_ vectors), it would not be beneficial.

- Try using the `#pragma omp simd` directive seen in class and the compiler option `-fopenmp-simd`.

See all the `bdot_opt*_simd` versions and related results (where `*` is 1,2,3). The addition of the `#pragma omp simd` directive only significantly helps the `bdot_opt3` optimized version, and leaves the `bdot_opt1` and `bdot_opt2` results essentially unaltered.

In [ ]:
! gcc -O3 -march=native -fopenmp-simd -fopenmp dot.c -o dot

In [ ]:
! OMP_NUM_THREADS=4 ./dot -r 10 -n 10000 -b

## Code:

```{literalinclude} ../c_codes/hw3_solution/dot.c
:language: c
:linenos: true
```